In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,datetime,time,math,itertools,os,sys

import torch
import torch.nn as nn
from torch.nn import functional as F

import tensorflow as tf

from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation,Dropout,BatchNormalization,Concatenate,Input
from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift
from sklearn.model_selection import train_test_split

import pandas as pd

import snowflake.connector

import seaborn as sns

C:\Users\z003mxpm\Anaconda3\lib\site-packages\snowflake\connector\options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (4.0.0), please install a version that adheres to: 'pyarrow<3.1.0,>=3.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [ ]:
query = '''
SELECT 
     acc."Account Name (MainAccount)"||'||'||acc."Account Country" AS "Key"
    ,olis."OLI.Close Date"
    ,SUM(olis."Funnel Value (GC)") as "Funnel Value (GC)"
FROM "CRMCL_TOPICAREA"."OSC"."Accounts" acc
LEFT JOIN "CRMCL_TOPICAREA"."OSC"."OLIs" olis ON acc."ACCOUNT_ID"=olis."ACCOUNT_ID"
WHERE acc."CKAP (Flag)"='Y'
AND olis."OLI.Close Date">TO_DATE('01-01-2012','DD-MM-YYYY')
AND olis."OLI.Close Date"<TO_DATE('01-01-2022','DD-MM-YYYY')
GROUP BY
"Key"
,olis."OLI.Close Date"
ORDER BY "Key",olis."OLI.Close Date" ASC
'''

df = pd.read_sql(query,con=conn)

In [2]:
conn = snowflake.connector.connect(
    user='jan-lucas.deinhard@siemens-healthineers.com',
    account='shsitdl.west-europe.azure',
    authenticator='externalbrowser'
)

In [3]:
info = pd.read_sql("SHOW COLUMNS",con=conn)

In [4]:
conn.close()

In [33]:
df = info[info['database_name']=='CRMCL_TOPICAREA']

df['table_name'] = df[['schema_name','table_name']].apply(lambda x: x[0]+'.'+x[1],axis=1)
df = df[df['schema_name'].isin([
    'AUXFILES',
    'CNJ',
    'CPQ',
    'GCR',
    'OSC',
    'VARIAN'
])].drop(columns=['data_type','null?','kind','expression','comment','database_name','autoincrement','default','schema_name'])

<ipython-input-33-b893168c8783>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['table_name'] = df[['schema_name','table_name']].apply(lambda x: x[0]+'.'+x[1],axis=1)


In [34]:
df = df.groupby(['column_name'],as_index=False).agg({'table_name':'||'.join})

In [40]:
df['num_tables_with_key'] = df['table_name'].apply(lambda x: len(str(x).split('||')))

In [43]:
df = df.sort_values(by='num_tables_with_key',ascending=False)

In [45]:
df[df['num_tables_with_key']>1]

,column_name,table_name,num_tables_with_key
1,ACCOUNT_ID,AUXFILES.Account_Address_Geolocations||AUXFILE...,54
1770,SECURITY_ID,CNJ.CNJ_MarketShare||CPQ.ACTIVE_QUOTES_HEADER|...,45
1658,Quote Number and Revision,CPQ.ACTIVE_QUOTES_HEADER||CPQ.CS_QUOTEITEM_COS...,22
996,Group Currency,CPQ.CS_QUOTEITEM_COSTS||CPQ.CS_Quote_Details||...,20
1405,PRODUCT_ID,CPQ.IN_VITRO_Consumables_per_Enduser||CPQ.IN_V...,17
...,...,...,...
1070,L1 Price (DC),CPQ.IN_VITRO_Price_Consumables||CPQ.IN_VITRO_P...,2
337,Base Price (BP) (DC),CPQ.CS_Quote_Details||CPQ.IN_VIVO_Quote_Details,2
1287,OLI.Record Eff Dt,OSC.OLI_HISTORY_SC_2016_2020||OSC.OLI_HISTORY_...,2
1245,Nr of Tests,CPQ.IN_VITRO_Price_Reagents||CPQ.IN_VITRO_Reag...,2
